In [ ]:
# %pip install -U langchain
# %pip install -qU langchain-community 
# beautifulsoup4
# Install package, compatible with API partitioning
# %pip install -qU langchain-community pypdf
# %pip install langchain unstructured[pdf] pdfminer.six
# pip install camelot-py[cv] pdfplumber pandas
# pip install sentence-transformers

In [ ]:
import os
import re
import pandas as pd
import pickle
from collections import defaultdict
import camelot
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

import dataloader

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import UnstructuredPDFLoader,PyPDFLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.faiss import DistanceStrategy

# Сбор данных

Итоговая реализация сохранена в `loader.py`

In [2]:
# website for parsing
url_isu = 'https://www.isu.org/figure-skating-rules/?tab=ISU%20Judging%20System'
# pdf for parsing
INPUT_PDF_PATH = 'pdf_rules'
PROF_PATH = os.path.join(INPUT_PDF_PATH,'Профессионалы')
LOVER_PATH = os.path.join(INPUT_PDF_PATH,'Любители')
ISU_PATH = os.path.join(INPUT_PDF_PATH,'ISU')

# all pdf files
prof_pdf_rules = os.listdir(PROF_PATH)
lover_pdf_rules = os.listdir(LOVER_PATH)
isu_pdf_rules = os.listdir(ISU_PATH)

prof_pdf_rules = [os.path.join(PROF_PATH, f)  for f in prof_pdf_rules]
lover_pdf_rules = [os.path.join(LOVER_PATH, f)  for f in lover_pdf_rules]
isu_pdf_rules = [os.path.join(ISU_PATH, f)  for f in isu_pdf_rules]

# общее количество правил в pdf
all_pdf = prof_pdf_rules + lover_pdf_rules + isu_pdf_rules; len(all_pdf)

28

## Веб-ресурсы

In [3]:
# Функция для извлечения информации с веб-сайтов (с сайта ISU)
def load_url(url):
    '''DataLoader for url
    Input: url
    Output: split'''
    
    loader_web = WebBaseLoader(url_isu)
    docs = loader_web.load()
     # разбиваем склеенные слова по заглавной букве
    # SustainabilityPressAnti-dopingSafeguardingISU  --> Sustainability Press Anti-doping Safeguarding ISU
    
    for i in range(len(docs)):
        s = docs[i].page_content 
        docs[i].page_content = re.sub(r'(?<=[a-z0-9])(?=[A-Z])', ' ', s)
    return docs

In [4]:
docs_web = load_url(url_isu)
docs_web[0].page_content

"Sports Rules - International Skating Union Homenews Events E-Rink Presshome of skating Governance History About ISUISU Constitution Structure Members Athletes Commission Transparencydown-arrow ISU Congresses Decision of the Council ISU Communications ISU Policies Strategy & Financial Reports Disciplinary Decisions Jobs Tenders & Bids Prevention of Manipulation of Competition Development Development Development Projects Visiting ISU Coach ISU Development Camps Scholarships Annual Contributions Center of excellence World Ice Skating Daye-Rink Olympic Solidarity Sports rules Sports rules Rule Change Proposal ISU Communications Anti-Doping Anti-Doping Rules Guidelines for Skaters PURE AS ICE - Education/Prevention Anti-Doping Program Guidelines for ISU Events Medical & Health ISU Medical Form Guidelines for ISU Events Health Harassment and abuse in sport Safeguarding Sustainability Homenews Events E-Rink Press Governance+Development+Sport Rules+anti-doping+Medical & Health+Safeguarding Su

## PDF

In [ ]:
pdf_file = prof_pdf_rules[1] # на примере одного файла

In [34]:
# извлекаем информацию из pdf
loader = UnstructuredPDFLoader(pdf_file, mode="paged") # постранично
page_docs = loader.load()        
num_pages = len(page_docs)

`mode='paged'` is deprecated in favor of the 'by_page' chunking strategy. Learn more about chunking here: https://docs.unstructured.io/open-source/core-functionality/chunking


In [35]:
# извлекаем табличные данные из pdf в структурированном виде
raw_tables = camelot.read_pdf(pdf_file, pages="all", flavor="lattice")  # ищет таблицы по линиям (сетке)
print(raw_tables.n)
raw_tables[0].df

18


,0,1,2,3
0,Статус спортивных \nсоревнований,"Пол, возраст",Спортивная дисциплина,Требование: занять место
1,1,2,3,4
2,Олимпийские зимние игры,"Мужчины, женщины","Одиночное катание,\nпарное катание, \nтанцы на...",1-8
3,Чемпионат мира,"Мужчины, женщины","Одиночное катание,\nпарное катание,\nтанцы на ...",1-7
4,,,Синхронное катание,1-6
5,Чемпионат Европы,"Мужчины, женщины","Одиночное катание,\nпарное катание,\nтанцы на ...",1-6


> Одна таблица может быть на нескольких страницах, при этом встречаются случаи, когда заголовки таблиц не переносятся на новую страницу. Требуется определить, является ли таблица продолжением предыдущей или это новая таблица. Для объединения таких таблиц в одну, будем считать что, если количество столбцов в таблицах на соседних страницах совпадает, то это одна и та же таблица.

Проверка, что это продолжение предыдущей таблицы:
- одинаковое число колонок
- страница сразу после предыдущей

In [36]:
def concat_tables(raw_tables):
    merged_tables = []   # список логических таблиц
    current = None       # текущая собираемая многостраничная таблица

    for t in raw_tables:
        df = t.df
        page = int(t.page)       # страницы, нумерация с 1
        ncols = df.shape[1]

        if current is None:  # инициализация первой таблицы
            # начинаем новую логическую таблицу
            current = {
                "df": df.copy(),
                "pages": [page],
                "ncols": ncols,
            }
            continue

        prev_page = current["pages"][-1] 
        prev_ncols = current["ncols"]

        # проверка, что это продолжение предыдущей таблицы:
        #   - такое же число колонок
        #   - страница сразу после предыдущей (prev_page + 1)
        if (ncols == prev_ncols) and (page == prev_page + 1):
            new_df = df.copy()

            # На случай, если заголовок всё‑таки повторяется на новой странице:
            # если первая строка нового куска == первой строке общей таблицы,
            # то считаем её дублирующим заголовком и выбрасываем
            # текущая таблица является продолжением предыдущей таблицы
            if (new_df.iloc[0] == current["df"].iloc[0]).all():
                new_df = new_df.iloc[1:]

            current["df"] = pd.concat([current["df"], new_df], ignore_index=True)   # объединяем их
            current["pages"].append(page)
        else:
            # предыдущая логическая таблица закончилась
            # на новой странице новая таблица
            merged_tables.append(current)
            current = {
                "df": df.copy(),
                "pages": [page],
                "ncols": ncols,
            }

    # последний элемент
    if current is not None:
        merged_tables.append(current)
        
    return merged_tables

In [37]:
merged_tables = concat_tables(raw_tables)
merged_tables  # собранные таблицы

[{'df':                                                     0  \
  0                    Статус спортивных \nсоревнований   
  1                                                   1   
  2                             Олимпийские зимние игры   
  3                                      Чемпионат мира   
  4                                                       
  5                                    Чемпионат Европы   
  6                                     Первенство мира   
  7                                                       
  8                                                       
  9                       Всемирные \nстуденческие игры   
  10  Другие международные \nспортивные соревнования...   
  11                                                      
  12                                                      
  13                                                      
  14                                       Иные условия   
  
                                               

> camelot называет таблицы по индексам столбцов, а не по первой строке [0,1,2,...]. Реально название таблицы находится в 1 строке. Переименуем таблицы в корректный вид
> Приведем таблицы к виду, удобному для поиска при реализации RAG, чтобы сохранились все соотношения: 
- Столбец 1: строка 1. Столбец 2: Строка 1, ... , Столбец n: строка 1
- Столбец 1: Строка 2, Столбец 2: Строка 2, ... , Столбец n : строка 2
- ...
- Столбец 1: строка m, Столбец 2: Строка m. ... , Столбец n: Строка m

In [38]:
#очистка заголовков 
def normalize_header_and_data(df: pd.DataFrame) -> pd.DataFrame:
    
    # заполняем пропуски для объединенных ячеек значениями из предыдущих строк
    df.fillna(method='ffill', inplace = True)   
    
    """Первая строка — заголовок, остальное — данные"""
    # Убираем полностью пустые строки/колонки
    df = df.dropna(axis=0, how="all")
    df = df.dropna(axis=1, how="all")

    if df.empty:
        return df

    # Берём первую строку как заголовок
    header = df.iloc[0].astype(str).str.strip()
    data = df.iloc[1:].copy()

    # Названия колонок
    data.columns = header

    # переводим все в стринговый формат и удаляем пробелы по краям строк
    data = data.applymap(lambda x: str(x).strip())

    # Убираем полностью пустые строки (все ячейки == "")
    mask_not_empty = ~data.apply(lambda r: all(v == "" for v in r), axis=1)
    data = data[mask_not_empty].reset_index(drop=True)

    return data


In [39]:
def df_to_rowwise_text(df: pd.DataFrame) -> str:
    """
    Превращает таблицу в текст:
    'Колонка1: значение; Колонка2: значение; ...'
    по одной строке на каждую запись.
    """
    headers = list(df.columns)
    lines = []

    for i, row in df.iterrows():
        parts = []
        for col in headers:
            value = str(row[col]).strip()
            if value:  # пропускаем пустые ячейки
                parts.append(f"{col}: {value}")
        if not parts:
            continue

        line = "; ".join(parts)  
        line = line.replace('\n',';')
        lines.append(line)

    return "\n".join(lines)

In [40]:
tables_text_by_page = defaultdict(list)

for idx, t in enumerate(merged_tables, start=1):
    df_raw = t["df"]  # отдельный датафрейм
    pages = t["pages"]          # список страниц, на которых тянется таблица
    start_page = pages[0]   # первая страницы, откуда начинается таблица
    
    df_clean = normalize_header_and_data(df_raw)  # преобразовываем названия 
    if df_clean.empty:
        continue

    table_text = df_to_rowwise_text(df_clean)   # таблица в текст
    
    tables_text_by_page[start_page].append(
        {
            "table_index": idx,
            "pages": pages,
            "text": table_text,
        }
    )

> В pdf файлах текст и таблицы идут друг за другом в хаотичном порядке. Соберем последовательно по страницам сначала текстовую информацию со страницы, потом табличную

Для каждой страницы:
   - текст страницы (как есть из Unstructured)
   - все таблицы, которые начинаются на этой странице

Приводим к виду, необходимому для загрузки в базу данных ([Docemunt]) и сохраняем метаданные для каждого документа

In [41]:
def clean_pdf_text(text: str) -> str:
    """
    Чистит PDF-текст от мусорных строк:
    - одиночные цифры и номера страниц
    - пустые строки
    - строки с dtype/object (артефакты таблиц)
    - строки состоящие только из пробелов, табов или пунктуации
    - отдельные мусорные столбцы таблиц
    """

    cleaned_lines = []
    
    for line in text.splitlines():
        original = line
        line = line.strip()

        # 1) Пустая строка -> удалить
        if not line:
            continue
        
        # 2) Строка содержит только цифру или номер страницы (1–3 символа)
        #    Пример: "1" "12" "141"
        if re.fullmatch(r"\d{1,4}", line):
            continue
        
        # 3) dtype/object, NaN, Series/Index артефакты от pandas
        if re.search(r"(dtype|Series|Name:|object)", line):
            continue
        
        # 4) Строки вида ":" или ": 0"
        if re.fullmatch(r":\s*\d*", line):
            continue
        
        # 5) Строки состоящие только из пунктуации или спецсимволов
        if re.fullmatch(r"[\W_]+", line):
            continue
        
        # 6) Строки с набором одиночных букв из таблиц (артефакты)
        # Например: "М Ж М Ж", "II III II III"
        if re.fullmatch(r"([IVXМЖ]\s*){2,}", line):
            continue
        
        # 7) Удаляем строки, где только уровень разряда без значения
        #    Например: "II", "III", "IV"
        if re.fullmatch(r"(I|II|III|IV|V|VI|VII|VIII|IX|X)$", line):
            continue
        
        # 8) Строки из односложных обрывков колонок
        if len(line) < 3:
            continue
        
        # 9) 1,2   1,12   12,12
        if re.fullmatch(r"\d{1,},\d{1,}", line):
            continue
        
        # 10) 1-2   1-12   12-15
        if re.fullmatch(r"\d{1,}-\d{1,}", line):
            continue
        
        # Если строка нормальная → вернуть
        cleaned_lines.append(original.strip())

    return "\n".join(cleaned_lines)


In [42]:
final_docs = []

for page_idx, page_doc in enumerate(page_docs): # то что извлекли из unstructedpdfloader
    page_number = page_idx + 1  # нумерация с 0, начинаем с 1

    parts = []

    # Текст страницы, очищаем от ненужных строк 
    raw_page_text = page_doc.page_content or ""
    page_text = raw_page_text.strip()    
    if page_text:
        parts.append(page_text)

    # Таблицы, начинающиеся на этой странице
    for tinfo in tables_text_by_page.get(page_number, []):
        header_line = (
            f"Таблица {tinfo['table_index']} "
            f"(страницы {', '.join(map(str, tinfo['pages']))}):"
        )
        parts.append(header_line)
        parts.append(tinfo["text"])

    # Если на странице вообще ничего нет (ни текста, ни таблиц) — пропускаем
    if not parts:
        continue

    combined_text = "\n\n".join(parts)
    combined_text_clean = clean_pdf_text(combined_text)
    # combined_text_clen =  clean_pdf_text(combined_text)
    # сохраняем метаданные
    metadata = dict(page_doc.metadata)
    # корректный номер страницы в метаданных
    metadata["page_number"] = page_number
    metadata.setdefault("source", pdf_file)

    final_docs.append(
        Document(
            page_content=combined_text_clean,
            metadata=metadata,
        )
    )


In [43]:
final_docs[:3]

[Document(metadata={'source': 'pdf_rules\\Профессионалы\\evsk_fs_2326_311024.pdf', 'coordinates': {'points': ((639.58, 494.7638400000001), (639.58, 507.00384), (655.89592, 507.00384), (655.89592, 494.7638400000001)), 'system': 'PixelSpace', 'layout_width': 841.68, 'layout_height': 595.2}, 'file_directory': 'pdf_rules\\Профессионалы', 'filename': 'evsk_fs_2326_311024.pdf', 'last_modified': '2025-11-24T01:26:34', 'page_number': 1, 'languages': ['rus'], 'filetype': 'application/pdf', 'parent_id': '47a7d7685bdde9ff6a4d060813549f13'}, page_content='Приложение № 11 к приказу Минспорта России от «_14» декабря_ 2022 г. № 1216\nС изменениями, внесенными приказом Минспорта России от 31.10.24. № 1076\nНормы, требования и условия их выполнения по виду спорта «фигурное катание на коньках»\n1. Требования и условия их выполнения для присвоения спортивного звания «мастер спорта России международного класса».\nМСМК выполняется с 14 лет, в спортивной дисциплине «синхронное катание» – с 15 лет\nСтатус сп

In [5]:
# для проверки сохраняем в .txt, чтобы посмотреть на результат
TXT_CHECK_DIRR = 'extract_pdf'
def save_txt_result(pdf_file, final_docs):
    txt_file = os.path.join(TXT_CHECK_DIRR, os.path.splitext(pdf_file.split('\\')[-1])[0] + ".txt")

    final_docs_sorted = sorted(final_docs, key=lambda d: d.metadata.get("page_number", 0))

    with open(txt_file, "w", encoding="utf-8") as f:
        for i, doc in enumerate(final_docs_sorted):
            if i > 0:
                f.write("\n\n\n")
            f.write(doc.page_content)


## PDF result

Загружаем и обрабатываем по такой логике все файлы PDF

In [6]:
all_docs = []
error_pdf_files = []
for pdf_file in tqdm(all_pdf):
    try:
        docs_pdf = dataloader.load_pdf(pdf_file)
        all_docs.extend(docs_pdf)
        save_txt_result(pdf_file, docs_pdf)  # посмотреть результат
    except:  
        error_pdf_files.append(pdf_file)
        # сохраняем документы, которые не обработались 
all_docs.extend(docs_web)

  0%|          | 0/28 [00:00<?, ?it/s]

`mode='paged'` is deprecated in favor of the 'by_page' chunking strategy. Learn more about chunking here: https://docs.unstructured.io/open-source/core-functionality/chunking
  4%|▎         | 1/28 [00:23<10:44, 23.86s/it]

`mode='paged'` is deprecated in favor of the 'by_page' chunking strategy. Learn more about chunking here: https://docs.unstructured.io/open-source/core-functionality/chunking
  7%|▋         | 2/28 [01:53<26:59, 62.31s/it]

`mode='paged'` is deprecated in favor of the 'by_page' chunking strategy. Learn more about chunking here: https://docs.unstructured.io/open-source/core-functionality/chunking
 11%|█         | 3/28 [03:49<36:11, 86.86s/it]

`mode='paged'` is deprecated in favor of the 'by_page' chunking strategy. Learn more about chunking here: https://docs.unstructured.io/open-source/core-functionality/chunking
 14%|█▍        | 4/28 [05:52<40:33, 101.38s/it]

...

 86%|████████▌ | 24/28 [18:38<00:52, 13.10s/it]`mode='paged'` is deprecated in favor of the 'by_page' chunking strategy. Learn more about chunking here: https://docs.unstructured.io/open-source/core-functionality/chunking


 89%|████████▉ | 25/28 [18:41<00:30, 10.15s/it]

`mode='paged'` is deprecated in favor of the 'by_page' chunking strategy. Learn more about chunking here: https://docs.unstructured.io/open-source/core-functionality/chunking
 93%|█████████▎| 26/28 [19:39<00:49, 24.50s/it]`mode='paged'` is deprecated in favor of the 'by_page' chunking strategy. Learn more about chunking here: https://docs.unstructured.io/open-source/core-functionality/chunking


 96%|█████████▋| 27/28 [20:17<00:28, 28.41s/it]`mode='paged'` is deprecated in favor of the 'by_page' chunking strategy. Learn more about chunking here: https://docs.unstructured.io/open-source/core-functionality/chunking


100%|██████████| 28/28 [20:20<00:00, 43.60s/it]


# Splitter + Vector Database

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200,    
                                               separators=["\n\n\n", "\n\n", "\n", " "],
)
splits = text_splitter.split_documents(all_docs); len(splits)

2804

In [ ]:
# Сохраняем список чанков в файл
with open('chunks.pkl', 'wb') as f:
    pickle.dump(splits, f)

In [ ]:
# # Загружаем чанки
# with open('chunks.pkl', 'rb') as f:
#     splits = pickle.load(f)

# print(f"Загружено {len(splits)} чанков")

Загружено 2804 чанков


In [ ]:
# мультиязычная модель эмбеддингов, тк правила на русском и на английском

model_name = "BAAI/bge-m3"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True, 'batch_size': 32}

embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

/tmp/ipython-input-30106893.py:7: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceBgeEmbeddings(


In [ ]:
batch_size = 100
vectorstore = None
for i in tqdm(range(0, len(splits), batch_size), desc="Векторизация"):
    batch = splits[i : i + batch_size]

    if vectorstore is None:
        # Создаем базу с первой пачкой
        vectorstore = FAISS.from_documents(
            batch,
            embeddings,
            distance_strategy=DistanceStrategy.COSINE
        )
    else:
        # Добавляем остальные пачки
        vectorstore.add_documents(batch)

Векторизация: 100%|██████████| 29/29 [02:28<00:00,  5.11s/it]


In [ ]:
folder_name = "faiss_index_bge_m3"
vectorstore.save_local(folder_name)

# RAG

In [ ]:
# загружаем базу ввекторную данных
vectorstore_db = FAISS.load_local(
    "faiss_index_bge_m3", 
    embeddings, 
    allow_dangerous_deserialization=True
)

In [ ]:
retriever = vectorstore_db.as_retriever(
    search_type="similarity",  # семантическая близость, косинусное расстояние
    # search_type = 'mmr',  # избегаение дублирования информации в ответе
    search_kwargs={"k": 5}  # 5 ближайших соседей
)

In [ ]:
print(retriever.invoke('С какого ребра выполняется прыжок флип/Flip?')[0].page_content)

Отрыв с неправильного ребра (Флип/Лутц)
Отрыв Флипа с заднего внутреннего ребра, отрыв Лутца с заднего наружного ребра. В случаях отсутствия отрыва с чистого правильного ребра Техническая бригада указывает судьям на наличие этой ошибки, используя знаки “e” (ребро) и “!” (внимание). Техническая Бригада может смотреть повтор в замедленной скорости. Техническая Бригада использует знак “e” (ребро), если ребро отрыва явно неправильно. Базовые стоимости прыжков со знаком “e” приведены в специальных строчках Шкалы Стоимости. Техническая Бригада использует знак “!”, если ребро отрыва не ясно. В этих случаях базовая стоимость не уменьшается. Обе эти ошибки отражаются судьями в своих GOE.
Недокрученные прыжки с отрывом с неправильного ребра (Флип/Лутц)
Если к одному и тому же прыжку применены оба знака “e” и “<”, то базовая стоимость приведена в специальных строчках Шкалы Стоимости.
Сорванный прыжок из Списка
